# Lorentz vectors

### 1. vector: Kinematic Vector Manipulation

The vector library is designed to work with Euclidean and Lorentz vectors (quadrivectors) in an efficient and natural way. It integrates seamlessly with NumPy and Awkward Array.

Individual Vectors. You can create and operate on individual vectors very easily.

In [2]:
import sys
import site
import numpy as np
import awkward as ak
import uproot
import matplotlib.pyplot as plt

# Get the user site-packages path
user_site_packages = site.getusersitepackages()

# Check if it's already in sys.path to avoid adding duplicates
if user_site_packages not in sys.path:
    sys.path.insert(0, user_site_packages) 

print("Updated sys.path:")
for p in sys.path:
    print(f"  - {p}")

# Now importing
import skhep_testdata
print("skhep_testdata imported successfully!")

Updated sys.path:
  - /eos/user/w/wbuitrag/.local/lib/python3.11/site-packages
  - /eos/home-i01/w/wbuitrag/SWAN_projects/PocketCoffea
  - /cvmfs/sft.cern.ch/lcg/releases/condor/24.0.7-acbb2/x86_64-el9-gcc13-opt/lib/python3
  - /cvmfs/sft.cern.ch/lcg/views/LCG_107_swan/x86_64-el9-gcc13-opt/lib/python3.11/site-packages/itk
  - /cvmfs/sft.cern.ch/lcg/views/LCG_107_swan/x86_64-el9-gcc13-opt/python
  - /cvmfs/sft.cern.ch/lcg/views/LCG_107_swan/x86_64-el9-gcc13-opt/lib
  - 
  - /cvmfs/sft.cern.ch/lcg/views/LCG_107_swan/x86_64-el9-gcc13-opt/lib/python3.11/site-packages
  - /usr/local/lib/swan/nb_term_lib
  - /cvmfs/sft.cern.ch/lcg/releases/Python/3.11.9-2924c/x86_64-el9-gcc13-opt/lib/python311.zip
  - /cvmfs/sft.cern.ch/lcg/releases/Python/3.11.9-2924c/x86_64-el9-gcc13-opt/lib/python3.11
  - /cvmfs/sft.cern.ch/lcg/releases/Python/3.11.9-2924c/x86_64-el9-gcc13-opt/lib/python3.11/lib-dynload
  - /cvmfs/sft.cern.ch/lcg/releases/Python/3.11.9-2924c/x86_64-el9-gcc13-opt/lib/python3.11/site-packag

In [3]:
import vector
import skhep_testdata, uproot
import awkward as ak

one = vector.obj(px=1, py=0, pz=0)
two = vector.obj(px=0, py=1, pz=1)

one + two

one.deltaR(two)

1.8011719796199461

#### one.to_rhophieta()

In [4]:
two.to_rhophieta()

MomentumObject3D(pt=1.0, phi=1.5707963267948966, eta=0.881373587019543)

Vectors with NumPy (Structured Arrays)

The true power of vectors is seen when applied to large data arrays. Here we use it with NumPy.

In [5]:
tree = uproot.open(skhep_testdata.data_path("uproot-Zmumu.root"))["events"]

In [6]:
one = ak.to_numpy(tree.arrays(filter_name=["E1", "p[xyz]1"]))
two = ak.to_numpy(tree.arrays(filter_name=["E2", "p[xyz]2"]))

In [7]:
one.dtype.names = ("E", "px", "py", "pz")
two.dtype.names = ("E", "px", "py", "pz")

In [8]:
one = one.view(vector.MomentumNumpy4D)
two = two.view(vector.MomentumNumpy4D)

In [9]:
one + two

MomentumNumpy4D([( -7.0508504 ,   1.31371932, -116.3919462 , 142.82374098),
                 ( -6.07723788,   0.86288157, -117.74020834, 144.54679534),
                 ( -5.76527367,   0.72893471, -117.22250173, 143.92770728), ...,
                 (-35.664423  , -24.9064416 , -226.76744871, 250.05025691),
                 (-36.41664408, -25.19899466, -228.38003444, 251.853268  ),
                 (-36.30874217, -25.19705013, -228.65597631, 252.14934978)],
                dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('t', '<f8')])

In [10]:
one.deltaR(two)

array([3.10530402, 3.10550819, 3.10547199, ..., 2.81363587, 2.81359933,
       2.81354694])

Vectors with Awkward Arrays (Jagged Data)

This is the most common use case in particle physics, where we have a variable number of particles per event.

In [11]:
one.to_rhophieta()

MomentumNumpy3D([(44.7322,  2.74126  , -1.21769), (38.8311, -0.440873 , -1.05139),
                 (38.8311, -0.440873 , -1.05139), ...,
                 (32.3997,  0.0370275, -1.57044), (32.3997,  0.0370275, -1.57044),
                 (32.5076,  0.0369644, -1.57078)],
                dtype=[('rho', '<f8'), ('phi', '<f8'), ('eta', '<f8')])

In [12]:
two.to_rhophieta()

MomentumNumpy3D([(37.7582, -0.441078, -1.05138), (44.7322,  2.74126 , -1.21769),
                 (44.3927,  2.7413  , -1.21776), ...,
                 (72.8781, -2.77524 , -1.4827 ), (73.6852, -2.77519 , -1.48227),
                 (73.6852, -2.77519 , -1.48227)],
                dtype=[('rho', '<f8'), ('phi', '<f8'), ('eta', '<f8')])

After vector.register_awkward() is called, "Momentum2D", "Momentum3D", "Momentum4D" are record names that Awkward Array will recognize to get all the vector functions.

In [13]:
vector.register_awkward()

In [14]:
tree = uproot.open(skhep_testdata.data_path("uproot-HZZ.root"))["events"]

In [15]:
array = tree.arrays(filter_name=["Muon_E", "Muon_P[xyz]"])

In [16]:
muons = ak.zip(
    {"px": array.Muon_Px, "py": array.Muon_Py, "pz": array.Muon_Pz, "E": array.Muon_E},
    with_name="Momentum4D",
)

In [17]:
mu1, mu2 = ak.unzip(ak.combinations(muons, 2))

In [18]:
mu1 + mu2

<MomentumArray4D [[{x: -15.2, y: -11, z: ..., ...}], ...] type='2421 * var ...'>

In [19]:
mu1.deltaR(mu2)

<Array [[2.95], [], [2.13], ..., [], [], []] type='2421 * var * float32'>

In [20]:
muons.to_rhophieta()

<MomentumArray3D [[{rho: 54.2, phi: ..., ...}, ...], ...] type='2421 * var ...'>

### 2. particle: The Particle Database

The particle library provides a comprehensive interface to the Particle Data Group (PDG) database. It allows you to access particle properties and work with their identifiers.

In [21]:
import particle
from hepunits import GeV

In [22]:
particle.Particle.findall("pi")

[<Particle: name="pi0", pdgid=111, mass=134.9768 ± 0.0005 MeV>,
 <Particle: name="pi+", pdgid=211, mass=139.57039 ± 0.00018 MeV>,
 <Particle: name="pi-", pdgid=-211, mass=139.57039 ± 0.00018 MeV>,
 <Particle: name="pi(2)(1670)0", pdgid=10115, mass=1670.6 + 2.9 - 1.2 MeV>,
 <Particle: name="pi(2)(1670)+", pdgid=10215, mass=1670.6 + 2.9 - 1.2 MeV>,
 <Particle: name="pi(2)(1670)-", pdgid=-10215, mass=1670.6 + 2.9 - 1.2 MeV>,
 <Particle: name="pi(1300)0", pdgid=100111, mass=1300 ± 100 MeV>,
 <Particle: name="pi(1300)+", pdgid=100211, mass=1300 ± 100 MeV>,
 <Particle: name="pi(1300)-", pdgid=-100211, mass=1300 ± 100 MeV>,
 <Particle: name="pi(1)(1400)0", pdgid=9000113, mass=None>,
 <Particle: name="pi(1)(1400)+", pdgid=9000213, mass=None>,
 <Particle: name="pi(1)(1400)-", pdgid=-9000213, mass=None>,
 <Particle: name="pi(1800)0", pdgid=9010111, mass=1810 + 9 - 11 MeV>,
 <Particle: name="pi(1)(1600)0", pdgid=9010113, mass=1645 + 40 - 17 MeV>,
 <Particle: name="pi(1800)+", pdgid=9010211, mass=

In [24]:
z_boson = particle.Particle.from_name("Z0")

In [25]:
z_boson.mass / GeV, z_boson.width / GeV

(91.188, 2.4955)

In [26]:
print(z_boson.describe())

Name: Z0             ID: 23           Latex: $Z^{0}$
Mass  = 91188.0 ± 2.0 MeV
Width = 2495.5 ± 2.3 MeV
Q (charge)        = 0       J (total angular) = 1.0      P (space parity) = None
C (charge parity) = None    I (isospin)       = None     G (G-parity)     = None
    Antiparticle name: Z0 (antiparticle status: Same)


In [27]:
particle.Particle.from_pdgid(111)

<Particle: name="pi0", pdgid=111, mass=134.9768 ± 0.0005 MeV>

In [28]:
particle.Particle.findall(
    lambda p: p.pdgid.is_meson and p.pdgid.has_strange and p.pdgid.has_charm
)

[<Particle: name="D(s)+", pdgid=431, mass=1968.35 ± 0.07 MeV>,
 <Particle: name="D(s)-", pdgid=-431, mass=1968.35 ± 0.07 MeV>,
 <Particle: name="D(s)*+", pdgid=433, mass=2112.2 ± 0.4 MeV>,
 <Particle: name="D(s)*-", pdgid=-433, mass=2112.2 ± 0.4 MeV>,
 <Particle: name="D(s2)*(2573)+", pdgid=435, mass=2569.1 ± 0.8 MeV>,
 <Particle: name="D(s2)*(2573)-", pdgid=-435, mass=2569.1 ± 0.8 MeV>,
 <Particle: name="D(s0)*(2317)+", pdgid=10431, mass=2317.8 ± 0.5 MeV>,
 <Particle: name="D(s0)*(2317)-", pdgid=-10431, mass=2317.8 ± 0.5 MeV>,
 <Particle: name="D(s1)(2536)+", pdgid=10433, mass=2535.11 ± 0.06 MeV>,
 <Particle: name="D(s1)(2536)-", pdgid=-10433, mass=2535.11 ± 0.06 MeV>,
 <Particle: name="D(s1)(2460)+", pdgid=20433, mass=2459.5 ± 0.6 MeV>,
 <Particle: name="D(s1)(2460)-", pdgid=-20433, mass=2459.5 ± 0.6 MeV>]

In [29]:
print(particle.PDGID(211).info())

A              None
J              0.0
L              0
S              0
Z              None
abspid         211
charge         1.0
has_bottom     False
has_charm      False
has_down       True
has_fundamental_anti False
has_strange    False
has_top        False
has_up         True
is_Qball       False
is_Rhadron     False
is_SUSY        False
is_baryon      False
is_diquark     False
is_dyon        False
is_excited_quark_or_lepton False
is_gauge_boson_or_higgs False
is_generator_specific False
is_hadron      True
is_lepton      False
is_meson       True
is_nucleus     False
is_pentaquark  False
is_quark       False
is_sm_gauge_boson_or_higgs False
is_sm_lepton   False
is_sm_quark    False
is_special_particle False
is_technicolor False
is_valid       True
j_spin         1
l_spin         1
s_spin         1
three_charge   3



In [30]:
pdgid = particle.Corsika7ID(11).to_pdgid()
particle.Particle.from_pdgid(pdgid)

<Particle: name="K+", pdgid=321, mass=493.677 ± 0.015 MeV>

### 3. fastjet: Jet Reconstruction

Fastjet is the Python interface to the standard library in HEP for jet reconstruction. It allows particles (traces, calorimeter clusters) to be grouped into jets.

In [31]:
vector.register_awkward()

In [32]:
picodst = uproot.open(
    "https://pivarski-princeton.s3.amazonaws.com/pythia_ppZee_run17emb.picoDst.root:PicoDst"
)

In [33]:
px, py, pz = ak.unzip(
    picodst.arrays(filter_name=["Track/Track.mPMomentum[XYZ]"], entry_stop=100)
)

In [34]:
probable_mass = particle.Particle.from_name("pi+").mass / GeV

In [35]:
pseudojets = ak.zip(
    {"px": px, "py": py, "pz": pz, "mass": probable_mass}, with_name="Momentum4D"
)

In [36]:
good_pseudojets = pseudojets[pseudojets.pt > 0.1]

In [37]:
import fastjet

In [38]:
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 1.0)

In [39]:
clusterseq = fastjet.ClusterSequence(good_pseudojets, jetdef)

#--------------------------------------------------------------------------
#                         FastJet release 3.4.2
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


In [40]:
clusterseq.inclusive_jets()

<MomentumArray4D [[{px: -0.0423, ...}, ..., {...}], ...] type='100 * var * ...'>

In [41]:
ak.num(good_pseudojets), ak.num(clusterseq.inclusive_jets())

(<Array [24, 19, 13, 36, 39, 48, ..., 57, 41, 35, 34, 20, 93] type='100 * int64'>,
 <Array [8, 6, 6, 7, 8, 9, 8, 7, ..., 9, 8, 7, 6, 7, 4, 11] type='100 * int64'>)

This fastjet package uses Vector to get coordinate transformations and all the Lorentz vector methods you’re accustomed to having in pseudo-jet objects. I used Particle to impute the mass of particles with only track-level information.